In [1]:
#ZADANIE 1.
#Rozpoznaj elementy odpowiedzialne za przechowywanie ofert dotyczących cen.
#Wykorzystując klasę WebScraping, zapisz dane w NoSQL a następnie dokonaj eksploracji danych nt. aktualnych cen.

#ZADANIE 2.
#Utwórz w bazie NoSQL kolekcję do pobierania ofert nieruchomości.

#ZADANIE 3.
#Utwórz klasę o nazwie WebScraping w której umieść metodę o nazwie skrapuj(). Metoda ta powinna zapisywać strony z nieruchomościami w bazie NoSQL, przyjmując za argument numer podstrony do skrapowania (2,3,4...). W tym punkcie można zrobić refaktoring klasy tworzonej w ramach modułu w ramach wcześniejszych laboratoriów.

#ZADANIE 4.
#Utwórz metodę atrybuty, która pobierze i wyświetli podstawowe informacje nt. nieruchomości z bazy NoSQL. Wywołaj metodę atrybuty dla wszystkich podstron.

#ZADANIE 5.
#Utwórz metodę o nazwie zapisz, które będzie zapisywała w pliku CSV atrybuty nowo pobieranych ofert nieruchomości.

#ZADANIE 6.
#Zmodyfikuj program, aby mógł zapisywać szczegóły dotyczące oferty nieruchomości (miejscowość etc.)

#ZADANIE 7.
#Utwórz metodę o nazwie zapiszJSON, w której informacje o ofertach nieruchomości będą zapisywane w tej strukturze, a następnie kierowane do bazy NoSQL.

In [2]:
# LAB 3
# ZADANIE 1 i 2
from pymongo import MongoClient
import requests
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
class WebScraping:
    headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15"}
    content=""
    db=np.nan
    def __init__(self): # konstruktor, wywoływany przy tworzeniu obiektu
        klient=MongoClient("mongodb://127.0.0.1:27017")
        self.db=klient['N5205_2024'] # łączy się z bazą lub tworzy bazę, jeżeli nie istnieje
    def pobierz(self,url):
        r=requests.get(url,headers=self.headers,verify=False)
        self.content=r.text
        strona={'url':url,'contents':r.text,'date':datetime.now()}
        rezultat=self.db.lab3.insert_one(strona) # wstawia do kolekcji lab3 w bazie N5205_2024
        print("Wstawiłem stronę",url,"z numerem",rezultat.inserted_id)
    def atrybuty(self,url):
        contents=self.db.lab3.find_one({'url':url})
        soup=BeautifulSoup(contents['contents'],'html.parser')
        
        ceny=soup.findAll('span',{'direction':'horizontal'})
        adresy=soup.findAll('p',{'data-testid':'advert-card-address'})
        cechy=soup.findAll('div',{'data-testid':'advert-card-specs-list'})
        for cena, adres, cecha in zip(ceny,adresy,cechy):
            print(cena.text.strip(),";",adres.text.strip(),";",cecha.text.strip())
    def zapisz(self,nazwaPliku):
        ceny_lista=[]
        adresy_lista=[]
        cechy_lista=[]
        miejscowosci_lista=[]
        wojewodztwa_lista=[]
        dane=self.db.lab3.find({})
        for d in dane:
            soup=BeautifulSoup(d['contents'],'html.parser')
            ceny=soup.findAll('span',{'direction':'horizontal'})
            adresy=soup.findAll('p',{'data-testid':'advert-card-address'})
            cechy=soup.findAll('div',{'data-testid':'advert-card-specs-list'})
            for adres, cena, cecha in zip(adresy,ceny,cechy):
                    adresy_lista.append(adres.text.strip())
                    ceny_lista.append(cena.text.strip())
                    cechy_lista.append(cecha.text.strip())
                    miejscowosci_lista.append(d['url'].split("/")[10].split("?")[0])
                    wojewodztwa_lista.append(adres.text.strip().split(",")[-1])
            # jak zapisać wszystko do df?
            df=pd.DataFrame({'adres':adresy_lista,'cena':ceny_lista,'cecha':cechy_lista,
                            'miejscowosc':miejscowosci_lista,'wojewodztwo':wojewodztwa_lista})
            df.to_csv(nazwaPliku+".csv",index=False,sep=";")
            df.to_excel(nazwaPliku+".xlsx",index=False)
    def zapiszJSON(self,nazwaPliku):
        ceny_lista=[]
        adresy_lista=[]
        cechy_lista=[]
        miejscowosci_lista=[]
        wojewodztwa_lista=[]
        dane=self.db.lab3.find({})
        for d in dane:
            soup=BeautifulSoup(d['contents'],'html.parser')
            ceny=soup.findAll('span',{'direction':'horizontal'})
            adresy=soup.findAll('p',{'data-testid':'advert-card-address'})
            cechy=soup.findAll('div',{'data-testid':'advert-card-specs-list'})
            for adres, cena, cecha in zip(adresy,ceny,cechy):
                    adresy_lista.append(adres.text.strip())
                    ceny_lista.append(cena.text.strip())
                    cechy_lista.append(cecha.text.strip())
                    miejscowosci_lista.append(d['url'].split("/")[10].split("?")[0])
                    wojewodztwa_lista.append(adres.text.strip().split(",")[-1])
            # jak zapisać wszystko do df?
            df=pd.DataFrame({'adres':adresy_lista,'cena':ceny_lista,'cecha':cechy_lista,
                            'miejscowosc':miejscowosci_lista,'wojewodztwo':wojewodztwa_lista})
            df.to_json(nazwaPliku+".json",orient='records')

In [3]:
w=WebScraping()
#w.pobierz("https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/lubuskie/swiebodzinski/zbaszynek/zbaszynek?limit=36&ownerTypeSingleSelect=ALL&by=DEFAULT&direction=DESC&viewType=listing/page=2")

/Users/U756317/Library/Python/3.9/lib/python/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.otodom.pl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Wstawiłem stronę https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/lubuskie/swiebodzinski/zbaszynek/zbaszynek?limit=36&ownerTypeSingleSelect=ALL&by=DEFAULT&direction=DESC&viewType=listing/page=2 z numerem 666dba0869917d118e73ac03


In [4]:
w.db.lab3.count_documents({})

1

In [5]:
dane=w.db.lab3.find({})
#for d in dane:
#    print(d['url'].split("/")[10].split("?")[0],len(d['contents']))
#    w.atrybuty(d['url'])

zbaszynek 334484


In [6]:
w.zapiszJSON("nieruchomości_289473")

In [8]:
w.zapisz("nieruchomości_289473")

In [9]:
df=pd.read_excel('nieruchomości_289473.xlsx')
print('liczba wierszy',df.shape[0])
print(df['miejscowosc'].groupby(df['miejscowosc']).count())
print(df['wojewodztwo'].groupby(df['wojewodztwo']).count())
print(df.head())

liczba wierszy 0
Series([], Name: miejscowosc, dtype: int64)
Series([], Name: wojewodztwo, dtype: int64)
Empty DataFrame
Columns: [adres, cena, cecha, miejscowosc, wojewodztwo]
Index: []


In [10]:
df

,adres,cena,cecha,miejscowosc,wojewodztwo
